In [ ]:
!pip install gym
!pip install matplotlib
!pip install numpy


In [ ]:
from IPython.display import display, Javascript

def click_connect():
    display(Javascript("""
        function ClickConnect(){
            console.log("Working");
            document.querySelector("colab-toolbar-button").click();
        }
        setInterval(ClickConnect, 60000);
    """))

click_connect()


<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pygame --upgrade
import pandas as pd
import gym
import matplotlib.pyplot as plt
import numpy as np
env = gym.make("Taxi-v3")
env.reset()
#env.render(mode='human')  # This will open a graphical window.
#env.step(env.action_space.sample()) # Take a random action
#env.render()


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


71

In [ ]:
print("State Space:", env.observation_space)
print("Action Space:", env.action_space)
env.step(env.action_space.sample()) # Take a random action
#env.render()
alpha = 0.1        # Learning rate
gamma = 0.6        # Discount factor
epsilon = 0.1      # Exploration rate

num_episodes = 1000  # Set the number of episodes, e.g., 1000
# Initialize the Q-table with zeros
decay_rate = 0.99  # Example decay rate; adjust as needed
evaluation_episodes = 100  # Number of episodes for evaluation
q_table = np.zeros((env.observation_space.n, env.action_space.n))
decay_rate1 = 0.98  # Adjust this value to control exploration decay





State Space: Discrete(500)
Action Space: Discrete(6)


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [ ]:
def navigate_to(env, start, target):
    # Basic navigation logic
    if start[0] < target[0]:
        return 1  # Move south
    elif start[0] > target[0]:
        return 0  # Move north
    elif start[1] < target[1]:
        return 2  # Move east
    elif start[1] > target[1]:
        return 3  # Move west
    return env.action_space.sample()  # otherwise random action


In [6]:
import numpy as np
import gym
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Initialize environment
env = gym.make("Taxi-v3")

# Hyperparameter ranges
learning_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
decay_rates = [0.95, 0.99, 0.999]
num_episodes_list = [1000, 2000, 3000, 4000, 5000]

# Store results
results = []
training_rewards_all = []
validation_rewards_all = []

# DQN Model Builder
def build_dqn_model(state_space_size, action_space_size, learning_rate):
    model = Sequential([
        Dense(24, input_shape=(state_space_size,), activation='relu'),
        Dense(24, activation='relu'),
        Dense(action_space_size, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# DQN Algorithm
def dqn(env, learning_rate=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01, num_episodes=1000):
    state_space_size = env.observation_space.n
    action_space_size = env.action_space.n
    model = build_dqn_model(state_space_size, action_space_size, learning_rate)

    training_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]  # Extract state from the tuple if returned
        state = to_categorical(state, num_classes=state_space_size).reshape(1, -1)  # One-hot encode and reshape
        total_reward = 0
        done = False

        while not done:
            # Choose action based on epsilon-greedy policy
            if np.random.rand() < epsilon:
                action = env.action_space.sample()  # Explore
            else:
                action = np.argmax(model.predict(state)[0])  # Exploit

            # Take action and observe result
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            if isinstance(next_state, tuple):
                next_state = next_state[0]  # Extract state from the tuple if returned
            next_state = to_categorical(next_state, num_classes=state_space_size).reshape(1, -1)  # One-hot encode next state

            # Get Q-value predictions and update Q-value for the chosen action
            target = reward
            if not done:
                target = reward + gamma * np.amax(model.predict(next_state)[0])
            target_f = model.predict(state)
            target_f[0][action] = target

            # Train model on updated Q-values
            model.fit(state, target_f, epochs=1, verbose=0)

            # Update state
            state = next_state

        # Update epsilon after each episode
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
        training_rewards.append(total_reward)

        # Print episode stats occasionally
        if episode % 100 == 0:
            print(f"Episode: {episode}, Total Reward: {total_reward}, Epsilon: {epsilon}")

    return training_rewards, model

# Hyperparameter Search Loop
for learning_rate in learning_rates:
    for decay_rate in decay_rates:
        for num_episodes in num_episodes_list:
            print(f"Training with Learning Rate: {learning_rate}, Decay Rate: {decay_rate}, Episodes: {num_episodes}")
            training_rewards, model = dqn(env, learning_rate=learning_rate, num_episodes=num_episodes)
            training_rewards_all.append(training_rewards)

            # Plot training results for the current learning rate
            plt.plot(training_rewards, label=f"LR={learning_rate}, Decay={decay_rate}, Episodes={num_episodes}")
            plt.xlabel('Episodes')
            plt.ylabel('Training Rewards')
            plt.title('Training Rewards vs. Episodes for Different Learning Rates')
            plt.legend()

plt.show()


KeyboardInterrupt: 

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

# Initialize environment
env = gym.make("Taxi-v3")

# Hyperparameter ranges
learning_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
decay_rates = [0.95, 0.99, 0.999]
num_episodes_list = [1000, 2000, 3000, 4000, 5000]


# Store results
results = []
training_rewards_all_dqn = []
training_rewards_all_pg = []
validation_rewards_all_dqn = []
validation_rewards_all_pg = []
def one_hot_encode(state, state_space_size):
    # One-hot encode the state
    one_hot = np.zeros(state_space_size)
    one_hot[state] = 1
    return one_hot.reshape(1, -1)

def dqn(env, learning_rate, decay_rate, num_episodes):
    state_space_size = env.observation_space.n  # Number of discrete states
    action_space_size = env.action_space.n

    # Create the model
    model = Sequential([
        Dense(24, input_dim=state_space_size, activation='relu'),
        Dense(24, activation='relu'),
        Dense(action_space_size, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    replay_memory = deque(maxlen=2000)
    gamma = 0.6
    epsilon = 1.0
    epsilon_min = 0.01
    epsilon_decay = decay_rate

    training_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        state = one_hot_encode(state, state_space_size)  # Convert state to one-hot encoded vector
        total_reward = 0
        done = False

        while not done:
            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values[0])

            next_state, reward, done, _ = env.step(action)
            next_state = one_hot_encode(next_state, state_space_size)  # Convert next state to one-hot encoded vector
            total_reward += reward

            # Store experience in replay memory
            replay_memory.append((state, action, reward, next_state, done))
            state = next_state

            # Train the model with a batch of experiences from the replay memory
            if len(replay_memory) > 64:
                batch = random.sample(replay_memory, 64)
                for s, a, r, ns, d in batch:
                    target = r if d else r + gamma * np.amax(model.predict(ns)[0])
                    target_f = model.predict(s)
                    target_f[0][a] = target
                    model.fit(s, target_f, epochs=1, verbose=0)

        epsilon = max(epsilon_min, epsilon * epsilon_decay)  # Update epsilon for exploration
        training_rewards.append(total_reward)

    return training_rewards, model

def policy_gradient(env, learning_rate, num_episodes):
    state_space_size = env.observation_space.n
    action_space_size = env.action_space.n

    model = Sequential([
        Dense(24, input_dim=state_space_size, activation='relu'),
        Dense(24, activation='relu'),
        Dense(action_space_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy')

    gamma = 0.99
    training_rewards = []
    num_episodes = int(num_episodes)


    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_space_size])
        episode_rewards = []
        episode_states, episode_actions = [], []
        done = False

        while not done:
            action_probs = model.predict(state)[0]
            action = np.random.choice(action_space_size, p=action_probs)
            next_state, reward, done, _ = env.step(action)

            episode_states.append(state)
            episode_actions.append(action)
            episode_rewards.append(reward)

            state = np.reshape(next_state, [1, state_space_size])

        total_reward = sum(episode_rewards)
        training_rewards.append(total_reward)

        discounted_rewards = np.zeros_like(episode_rewards, dtype=np.float32)
        cumulative = 0
        for i in reversed(range(len(episode_rewards))):
            cumulative = cumulative * gamma + episode_rewards[i]
            discounted_rewards[i] = cumulative
        discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-7)

        for s, a, dr in zip(episode_states, episode_actions, discounted_rewards):
            action_vector = np.zeros(action_space_size)
            action_vector[a] = dr
            model.fit(s, np.array([action_vector]), verbose=0)

    return training_rewards, model

# Hyperparameter Search Loop for both DQN and Policy Gradient
for learning_rate in learning_rates:
    for decay_rate in decay_rates:
        for num_episodes in num_episodes_list:
            # DQN
            training_rewards_dqn, model_dqn = dqn(env, learning_rate, decay_rate, num_episodes)
            training_rewards_all_dqn.append(training_rewards_dqn)

            validation_total_rewards_dqn = []
            validation_episodes = 100
            for _ in range(validation_episodes):
                state = env.reset()
                state = np.array([state])
                total_reward = 0
                done = False
                while not done:
                    q_values = model_dqn.predict(state)
                    action = np.argmax(q_values[0])

                    next_state, reward, done, _ = env.step(action)
                    state = np.array([next_state])
                    total_reward += reward
                validation_total_rewards_dqn.append(total_reward)

            average_validation_reward_dqn = np.mean(validation_total_rewards_dqn)
            validation_rewards_all_dqn.append(average_validation_reward_dqn)
            results.append(('DQN', learning_rate, decay_rate, num_episodes, average_validation_reward_dqn))
           # print(f"DQN - Learning Rate: {learning_rate}, Decay Rate: {decay_rate}, Episodes: {num_episodes}, Average Validation Reward: {average_validation_reward_dqn}")

            # Policy Gradient
            training_rewards_pg, model_pg = policy_gradient(env, learning_rate, num_episodes)
            training_rewards_all_pg.append(training_rewards_pg)

            validation_total_rewards_pg = []
            for _ in range(validation_episodes):
                state = env.reset()
                state = np.reshape(state, [1, state_space_size])
                total_reward = 0
                done = False
                while not done:
                    action_probs = model_pg.predict(state)[0]
                    action = np.argmax(action_probs)
                    next_state, reward, done, _ = env.step(action)
                    state = np.reshape(next_state, [1, state_space_size])
                    total_reward += reward
                validation_total_rewards_pg.append(total_reward)

            average_validation_reward_pg = np.mean(validation_total_rewards_pg)
            validation_rewards_all_pg.append(average_validation_reward_pg)
            results.append(('Policy Gradient', learning_rate, decay_rate, num_episodes, average_validation_reward_pg))
           # print(f"Policy Gradient - Learning Rate: {learning_rate}, Decay Rate: {decay_rate}, Episodes: {num_episodes}, Average Validation Reward: {average_validation_reward_pg}")

# Save results and plot
results_df = pd.DataFrame(results, columns=["Method", "Learning Rate", "Decay Rate", "Episodes", "Average Validation Reward"])
results_df.to_csv("taxi_v3_results.csv", index=False)
np.save("taxi_v3_results.npy", np.array(results))

plt.figure(figsize=(12, 6))
for i, rewards in enumerate(training_rewards_all_dqn):
    plt.plot(rewards, label=f'DQN Config {i+1}')
for i, rewards in enumerate(training_rewards_all_pg):
    plt.plot(rewards, label=f'Policy Gradient Config {i+1}')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Training Reward for DQN and Policy Gradient')
plt.legend()
plt.grid()
plt.savefig("training_rewards_over_episodes.png")
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(validation_rewards_all_dqn, marker='o', linestyle='-', color='b', label='DQN')
plt.plot(validation_rewards_all_pg, marker='x', linestyle='-', color='r', label='Policy Gradient')
plt.xlabel('Configuration Index')
plt.ylabel('Average Validation Reward')
plt.title('Average Validation Reward Over Configurations')
plt.legend()
plt.grid()
plt.savefig("average_validation_reward.png")
plt.show()


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import os
import warnings
import sys
import os
import gym

# Redirect stdout and stderr to null to suppress printing
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')




warnings.filterwarnings("ignore", category=DeprecationWarning)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs
# Initialize environment
env = gym.make("Taxi-v3")
# Initialize environment with new_step_api set to True
#env = gym.make("Taxi-v3", new_step_api=True)

# PG Hyperparameters
learning_rates = [0.1]
num_episodes_list = [1000]
# PG Hyperparameters
#learning_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
#num_episodes_list = [1000, 2000, 3000, 4000, 5000]
training_rewards_all_pg = []
validation_rewards_all_pg = []
results = []
"""
def policy_gradient(env, learning_rate, num_episodes):
    state_space_size = env.observation_space.n
    action_space_size = env.action_space.n
    model = Sequential([
        Dense(24, input_dim=state_space_size, activation='relu'),
        Dense(24, activation='relu'),
        Dense(action_space_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy')

    gamma = 0.99
    training_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_space_size])
        episode_rewards, episode_states, episode_actions = [], [], []
        done = False
        while not done:
            action_probs = model.predict(state)[0]
            action = np.random.choice(action_space_size, p=action_probs)
            next_state, reward, done, _ = env.step(action)
            episode_states.append(state)
            episode_actions.append(action)
            episode_rewards.append(reward)
            state = np.reshape(next_state, [1, state_space_size])

        total_reward = sum(episode_rewards)
        training_rewards.append(total_reward)

        discounted_rewards = np.zeros_like(episode_rewards, dtype=np.float32)
        cumulative = 0
        for i in reversed(range(len(episode_rewards))):
            cumulative = cumulative * gamma + episode_rewards[i]
            discounted_rewards[i] = cumulative
        discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-7)
        for s, a, dr in zip(episode_states, episode_actions, discounted_rewards):
            action_vector = np.zeros(action_space_size)
            action_vector[a] = dr
            model.fit(s, np.array([action_vector]), verbose=0)
    return training_rewards, model
"""

# Updated policy gradient function
def policy_gradient(env, learning_rate, num_episodes):
    state_space_size = env.observation_space.n
    action_space_size = env.action_space.n

    model = Sequential([
        tf.keras.layers.Input(shape=(1,)),  # Set the input shape to match integer state
        Dense(24, activation='relu'),
        Dense(24, activation='relu'),
        Dense(action_space_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy')

    gamma = 0.99
    training_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        episode_rewards, episode_states, episode_actions = [], [], []
        done = False
        while not done:
            state_encoded = np.array([[state]])  # Keep state as a single integer
            action_probs = model.predict(state_encoded)[0]
            action = np.random.choice(action_space_size, p=action_probs)
            next_state, reward, done, _ = env.step(action)

            # Append necessary values
            episode_states.append(state_encoded)
            episode_actions.append(action)
            episode_rewards.append(reward)

            state = next_state  # Update state

        total_reward = sum(episode_rewards)
        training_rewards.append(total_reward)

        # Calculate discounted rewards and normalize them
        discounted_rewards = np.zeros_like(episode_rewards, dtype=np.float32)
        cumulative = 0
        for i in reversed(range(len(episode_rewards))):
            cumulative = cumulative * gamma + episode_rewards[i]
            discounted_rewards[i] = cumulative
        discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-7)

        # Train the model with the episode's states, actions, and rewards
        for state_encoded, action, dr in zip(episode_states, episode_actions, discounted_rewards):
            action_vector = np.zeros(action_space_size)
            action_vector[action] = dr
            model.fit(state_encoded, np.array([action_vector]), verbose=0)

    return training_rewards, model

# Hyperparameter Search
for lr in learning_rates:
    for num_episodes in num_episodes_list:
        training_rewards_pg, model_pg = policy_gradient(env, lr, num_episodes)
        training_rewards_all_pg.append(training_rewards_pg)
        validation_total_rewards_pg = []
        for _ in range(100):
            state = env.reset()
            state = np.reshape(state, [1, env.observation_space.n])
            total_reward = 0
            done = False
            while not done:
                action_probs = model_pg.predict(state)[0]
                action = np.argmax(action_probs)
                next_state, reward, done, _ = env.step(action)
                state = np.reshape(next_state, [1, env.observation_space.n])
                total_reward += reward
            validation_total_rewards_pg.append(total_reward)
        avg_validation_reward_pg = np.mean(validation_total_rewards_pg)
        validation_rewards_all_pg.append(avg_validation_reward_pg)
        results.append(('Policy Gradient', lr, num_episodes, avg_validation_reward_pg))

# Save results and plot
results_df = pd.DataFrame(results, columns=["Method", "Learning Rate", "Episodes", "Avg Validation Reward"])
results_df.to_csv("pg_taxi_v3_results.csv", index=False)
plt.plot(validation_rewards_all_pg, marker='x', linestyle='-', color='r', label='Policy Gradient')
plt.xlabel('Configuration Index')
plt.ylabel('Average Validation Reward')
plt.legend()
plt.savefig("pg_average_validation_reward.png")
plt.show()
# Reset stdout and stderr back to default after the code execution
sys.stdout = sys.__stdout__
sys.stderr = sys.__stderr__

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random
import os
import warnings
import sys
import os
import gym

# Redirect stdout and stderr to null to suppress printing
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')




warnings.filterwarnings("ignore", category=DeprecationWarning)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs

# Initialize environment
env = gym.make("Taxi-v3")

# DQN Hyperparameters
learning_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
decay_rates = [0.95, 0.99, 0.999]
num_episodes_list = [1000, 2000, 3000, 4000, 5000]
training_rewards_all_dqn = []
validation_rewards_all_dqn = []
results = []

def one_hot_encode(state, state_space_size):
    one_hot = np.zeros(state_space_size)
    one_hot[state] = 1
    return one_hot.reshape(1, -1)

def dqn(env, learning_rate, decay_rate, num_episodes):
    state_space_size = env.observation_space.n
    action_space_size = env.action_space.n

    model = Sequential([
        Dense(24, input_dim=state_space_size, activation='relu'),
        Dense(24, activation='relu'),
        Dense(action_space_size, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    replay_memory = deque(maxlen=2000)
    gamma = 0.6
    epsilon = 1.0
    epsilon_min = 0.01
    epsilon_decay = decay_rate

    training_rewards = []
    for episode in range(num_episodes):
        state = env.reset()
        state = one_hot_encode(state, state_space_size)
        total_reward = 0
        done = False
        while not done:
            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values[0])
            next_state, reward, done, _ = env.step(action)
            next_state = one_hot_encode(next_state, state_space_size)
            total_reward += reward
            replay_memory.append((state, action, reward, next_state, done))
            state = next_state
            if len(replay_memory) > 64:
                batch = random.sample(replay_memory, 64)
                for s, a, r, ns, d in batch:
                    target = r if d else r + gamma * np.amax(model.predict(ns)[0])
                    target_f = model.predict(s)
                    target_f[0][a] = target
                    model.fit(s, target_f, epochs=1, verbose=0)
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        training_rewards.append(total_reward)
    return training_rewards, model

# Hyperparameter Search
for lr in learning_rates:
    for dr in decay_rates:
        for num_episodes in num_episodes_list:
            training_rewards_dqn, model_dqn = dqn(env, lr, dr, num_episodes)
            training_rewards_all_dqn.append(training_rewards_dqn)
            validation_total_rewards_dqn = []
            for _ in range(100):
                state = env.reset()
                state = np.array([state])
                total_reward = 0
                done = False
                while not done:
                    q_values = model_dqn.predict(state)
                    action = np.argmax(q_values[0])
                    next_state, reward, done, _ = env.step(action)
                    state = np.array([next_state])
                    total_reward += reward
                validation_total_rewards_dqn.append(total_reward)
            avg_validation_reward_dqn = np.mean(validation_total_rewards_dqn)
            validation_rewards_all_dqn.append(avg_validation_reward_dqn)
            results.append(('DQN', lr, dr, num_episodes, avg_validation_reward_dqn))

# Save results and plot
results_df = pd.DataFrame(results, columns=["Method", "Learning Rate", "Decay Rate", "Episodes", "Avg Validation Reward"])
results_df.to_csv("dqn_taxi_v3_results.csv", index=False)
plt.plot(validation_rewards_all_dqn, marker='o', linestyle='-', color='b', label='DQN')
plt.xlabel('Configuration Index')
plt.ylabel('Average Validation Reward')
plt.legend()
plt.savefig("dqn_average_validation_reward.png")
plt.show()

# Reset stdout and stderr back to default after the code execution
sys.stdout = sys.__stdout__
sys.stderr = sys.__stderr__

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


KeyboardInterrupt: 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random
import os

# Set TensorFlow logging level to suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Initialize environment
env = gym.make("Taxi-v3")
env._max_episode_steps = 100  # Set a custom step limit, e.g., 100 steps


# DQN Hyperparameters
learning_rates = [0.1, 0.3, 0.5]
decay_rates = [0.95, 0.99]
num_episodes_list = [1000]
training_rewards_all_dqn = []
validation_rewards_all_dqn = []
results = []

def one_hot_encode(state, state_space_size):
    one_hot = np.zeros(state_space_size)
    one_hot[state] = 1
    return one_hot.reshape(1, -1)

def dqn(env, learning_rate, decay_rate, num_episodes):
    state_space_size = env.observation_space.n
    action_space_size = env.action_space.n


    # Simplified network
    model = Sequential([
        Dense(16, input_dim=state_space_size, activation='relu'),
        Dense(16, activation='relu'),
        Dense(action_space_size, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    replay_memory = deque(maxlen=1000)
    gamma = 0.6
    epsilon = 1.0
    epsilon_min = 0.01
    epsilon_decay = decay_rate

    training_rewards = []
    for episode in range(num_episodes):
        state = env.reset()
        state = one_hot_encode(state, state_space_size)
        total_reward = 0
        done = False
        while not done:
            # Epsilon-greedy action selection
            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(state, verbose=0)
                action = np.argmax(q_values[0])

            # Take action and store transition in replay memory
            next_state, reward, done, _ = env.step(action)
            next_state = one_hot_encode(next_state, state_space_size)
            total_reward += reward
            replay_memory.append((state, action, reward, next_state, done))
            state = next_state

            # Sample batch and train
            if len(replay_memory) > 32:
                batch = random.sample(replay_memory, 32)
                states = np.array([s[0].flatten() for s in batch])
                next_states = np.array([s[3].flatten() for s in batch])

                q_values = model.predict(states, verbose=0)
                q_values_next = model.predict(next_states, verbose=0)
                for i, (s, a, r, ns, d) in enumerate(batch):
                    target = r if d else r + gamma * np.amax(q_values_next[i])
                    q_values[i][a] = target
                model.fit(states, q_values, epochs=1, verbose=0)

        # Decay epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        training_rewards.append(total_reward)
    return training_rewards, model

# Hyperparameter Search
for lr in learning_rates:
    for dr in decay_rates:
        for num_episodes in num_episodes_list:
            training_rewards_dqn, model_dqn = dqn(env, lr, dr, num_episodes)
            training_rewards_all_dqn.append(training_rewards_dqn)
            validation_total_rewards_dqn = []
            for _ in range(100):
                state = env.reset()
                state = one_hot_encode(state, env.observation_space.n)
                total_reward = 0
                done = False
                while not done:
                    q_values = model_dqn.predict(state, verbose=0)
                    action = np.argmax(q_values[0])
                    next_state, reward, done, _ = env.step(action)
                    state = one_hot_encode(next_state, env.observation_space.n)
                    total_reward += reward
                validation_total_rewards_dqn.append(total_reward)
            avg_validation_reward_dqn = np.mean(validation_total_rewards_dqn)
            validation_rewards_all_dqn.append(avg_validation_reward_dqn)
            results.append(('DQN', lr, dr, num_episodes, avg_validation_reward_dqn))

# Save results and plot
results_df = pd.DataFrame(results, columns=["Method", "Learning Rate", "Decay Rate", "Episodes", "Avg Validation Reward"])
results_df.to_csv("dqn_taxi_v3_results.csv", index=False)
plt.plot(validation_rewards_all_dqn, marker='o', linestyle='-', color='b', label='DQN')
plt.xlabel('Configuration Index')
plt.ylabel('Average Validation Reward')
plt.legend()
plt.savefig("dqn_average_validation_reward.png")
plt.show()


KeyboardInterrupt: 